In [2]:
# импортирую библиотеки
import pandas as pd
from tqdm import tqdm

In [3]:
# загружаю ГБД ФЛ в датафрейм
df = pd.read_csv('data/inter/gbd_fl.csv', index_col=0, parse_dates=['birth_date', 'death_date'])
df

,birth_date,sex,nationality,citizenship,death_date,life_status,birth_country,birth_oblast,birth_rayon,birth_town,address_country,address_oblast,address_rayon,address_town,address_temp_country,address_temp_oblast,address_temp_rayon,address_temp_town,work_status,lifetime
iin,,,,,,,,,,,,,,,,,,,,
0,1937-07-17,2,3,1,2020-12-06,2,1,12,15,3,1,12,15,3,1,12,15,3,1,30458.0
1,1958-09-18,1,1,1,2002-02-18,2,1,5,9,1,1,5,9,1,1,5,9,1,1,15859.0
2,1955-01-20,1,1,1,2021-05-16,2,1,13,6,6,1,13,6,6,1,13,6,6,2,24223.0
3,2004-06-18,1,1,3,NaT,1,1,3,7,8,1,3,7,8,1,3,7,8,2,NaN
4,1927-02-18,1,1,1,1992-02-14,2,3,6,13,2,3,6,13,2,3,6,13,2,2,23737.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1990-01-19,2,1,1,NaT,1,1,7,18,6,1,7,18,6,1,7,18,6,2,NaN
499996,1921-06-22,1,2,3,1949-10-02,2,1,11,15,9,1,11,15,9,1,11,15,9,1,10329.0
499997,1960-10-28,1,1,1,2010-03-22,2,1,7,19,6,1,7,19,6,1,7,19,6,2,18042.0


Расчет коэффициентов осуществляется за месяц и за любое число месяцев с накоплением (период),с применением коэффициента перевода и средней численности населения.

Коэффициент перевода в месяце (периоде) вычисляется  с точностью до шестого знака после запятой по следующей формуле:

$$K_{t} = \frac{DN}{dn},$$
где:
- $K_t$ – коэффициент перевода в месяце (периоде);
- $DN$ – число дней в году;
- $dn$ – число дней в месяце (периоде);
- $t$ – месяц (период).

С помощью коэффициента перевода данные в месяце (периоде) приводятся к годовому выражению. Число дней в году берется  из расчета 365 дней (в високосном году – 366).

Средняя численность населения в месяце (периоде) вычисляется с точностью до одного знака после запятой по следующей формуле:

$$P^{t} = P^{1} + \frac{OP^{t}}{2},$$
где:
- $P^{t}$ – средняя численность населения;
- $P^{1}$ – численность населения на первое января отчетного года;
- $OP^{t}$ – общий прирост, убыль;
- $t$ – месяц (период).

Общий коэффициент смертности за месяц (период) рассчитывается по следующей формуле:

$$K(m^t) = \frac{M^t \cdot K^{t}}{P^{t}} \cdot 1000,$$
где:
- $K(m^t)$ – коэффициент смертности;
- $M^t$ – число умерших;
- $K^t$ – коэффициент перевода;
- $P^{t}$ – средняя численность населения;
- $t$ – месяц (период).

In [5]:
def death_coefficient_by_month(df, year, month):
    """
    Входные данные:
        df: датафрейм с физлицами
        month, year: год и месяц для которого
        рассчитывается статистика
    Выходные данные:
        Kmt: коэффициент смертности за месяц month
        Pt: Средняя численность населения в месяце
    """
    # true если год year високосный
    leap =  year % 400 == 0 or \
           (year % 100 != 0 and \
            year % 4 == 0)
            
    days_in_month = {
        1:  31, 2:  29 if leap else 28,
        3:  31, 4:  30,
        5:  31, 6:  30,
        7:  31, 8:  31,
        9:  30, 10: 31,
        11: 30, 12: 31, 
    }

    days_in_year = 366 if leap else 365
        
    Kt = round(days_in_year / days_in_month[month], 6)

    def population(month, day):
        P = df[(df.birth_date <= pd.Timestamp(year, month, day)) &
              ((df.death_date >  pd.Timestamp(year, month, day)) |
                df.life_status == 1)].shape[0]
        return P

    # средняя численность населения на 1 января
    P1 = population(1, 1)

    # общий прирост населения
    OP = population(month, 1) - population(1, 1)

    # средняя численность населения в месяце
    Pt = round(P1 + OP / 2, 1)

    # число умерших в интервале
    Mt = df[(df.death_date >= pd.Timestamp(year, month, 1)) & 
            (df.death_date <  pd.Timestamp(year, month, days_in_month[month]))].shape[0]

    # коэффициент смертности за месяц month
    Kmt = Mt * Kt / Pt * 1000

    return Kmt, Pt

In [6]:
def death_coefficients_df(df, year):
    """
    Входные данные:
        df: датафрейм с физлицами
        year: год для которого рассчитывается статистика
    Выходные данные:
        df_death_coefficients: датафрейм со статистиками
    """
    death_coefficients = {
        'month_id': [],
        'month':    [],
        'Kmt':      [],
        'Pt':       [],
    }

    # переводит номер месяца в название
    month_names = {
        1:  'january',      2:  'february',
        3:  'march',        4:  'april',
        5:  'may',          6:  'june',
        7:  'july',         8:  'august',
        9:  'september',    10: 'october',
        11: 'november',     12: 'december'
    }

    for month_id, month_name in month_names.items():
        Kmt, Pt = death_coefficient_by_month(df, year, month_id)
        death_coefficients['month_id'].append(month_id)
        death_coefficients['month'].append(month_name)
        death_coefficients['Kmt'].append(Kmt)
        death_coefficients['Pt'].append(Pt)

    df_death_coefficients = pd.DataFrame.from_dict(death_coefficients).set_index('month_id')

    return df_death_coefficients

death_coefficients_df(df, 2020)


,month,Kmt,Pt
month_id,,,
1,january,9.226369,490103.0
2,february,9.601241,490303.0
3,march,9.170742,490501.0
4,april,9.273700,490699.5
5,may,8.513774,490908.5
6,june,9.712960,491117.0
7,july,9.323239,491342.5
8,august,9.151329,491541.5
9,september,9.849264,491752.5


In [7]:
def save_death_coefficients(df, start_year, end_year):
    writer = pd.ExcelWriter(
        f'results/death_coefficients.xlsx', engine='xlsxwriter'
    )
    years = range(start_year, end_year)
    for year in tqdm(years):
    # сохраняю краткую таблицу смертности в формате .xlsx
        death_coefficients_df(df, year).to_excel(writer, sheet_name=f'{year}')
    writer.save()

In [8]:
save_death_coefficients(df, 1950, 2020)

100%|██████████| 70/70 [01:44<00:00,  1.49s/it]
